In [1]:
import numpy as np
import pandas as pd

import os
import shutil

from glob import glob 
from skimage.io import imread
import gc

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from DataGenerator import CreateData

Using TensorFlow backend.
C:\Users\dzhal\Anaconda3\envs\neural_network\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dzhal\Anaconda3\envs\neural_network\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dzhal\Anaconda3\envs\neural_network\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dzhal\Anaco

In [2]:
createData = CreateData()
createData.create_directory()

C:\Users\dzhal\informtelecs\DataGenerator.py:151: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(15, 10))  # think about scale
C:\Users\dzhal\informtelecs\DataGenerator.py:85: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(15, 10))
C:\Users\dzhal\informtelecs\DataGenerator.py:82: RuntimeWarning: divide by zero encountered in double_scalars
  y = 1/koef * np.array(x) if i == 0 else koef * np.array(x)
C:\Users\dzhal\informtelecs\DataGenerator.py:106: RuntimeWarning: More than 20 figures have been opened. Figu

In [50]:
smth_base = "D:informconnection\\"
base_tile_dir = smth_base + 'train\\*'
df = pd.DataFrame({'path': glob(os.path.join(base_tile_dir, '*.jpg'))})
df['target'] = df['path'].apply(lambda x: x.split('\\')[2])

In [69]:
np.unique(df['target'])

2091

In [64]:
import PIL
from PIL import Image
def external_scale(x):
    image = Image.open(x)
    img = image.resize((200, 200), PIL.Image.ANTIALIAS)
    img.save(x)

In [65]:
from collections import Counter
df['path'].apply(lambda x: external_scale(x))


0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
2061    None
2062    None
2063    None
2064    None
2065    None
2066    None
2067    None
2068    None
2069    None
2070    None
2071    None
2072    None
2073    None
2074    None
2075    None
2076    None
2077    None
2078    None
2079    None
2080    None
2081    None
2082    None
2083    None
2084    None
2085    None
2086    None
2087    None
2088    None
2089    None
2090    None
Name: path, Length: 2091, dtype: object

In [58]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from keras_radam import RAdam

kernel_size = (3,3)
pool_size= (2,2)
first_filters = 16
second_filters = 32

dropout_conv = 0.3
dropout_dense = 0.5

model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (200, 200, 3)))
model.add(Conv2D(first_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

#model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(64, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(4, activation = "softmax"))

# Compile the model
model.compile(RAdam(0.01), loss = "binary_crossentropy", metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 198, 198, 16)      448       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 196, 196, 16)      2304      
_________________________________________________________________
batch_normalization_17 (Batc (None, 196, 196, 16)      64        
_________________________________________________________________
activation_17 (Activation)   (None, 196, 196, 16)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 98, 98, 16)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 98, 98, 16)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 96, 96, 32)        4608      
__________

In [47]:
import tensorflow as tf

In [48]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [95]:
BATCH_SIZE = 32
CLASS_NAMES = ['exp', 'tan', 'sin_cos', 'linear']
STEPS_PER_EPOCH = np.ceil(len(df)/BATCH_SIZE)
train_data_gen = image_generator.flow_from_directory(directory="D:informconnection\\train",
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(200, 200),
                                                     classes = CLASS_NAMES)

Found 2091 images belonging to 4 classes.


In [88]:
import matplotlib.pyplot as plt
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
#       plt.title(CLASS_NAMES[label_batch[n]])
      plt.axis('off')
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

C:\Users\dzhal\Anaconda3\envs\neural_network\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


In [92]:
plt.plot([1, 2])

In [97]:
model.fit_generator(
        train_data_gen,
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
66/66 [==============================] - 43s 648ms/step - loss: 0.1658 - acc: 0.9377
Epoch 2/10
66/66 [==============================] - 12s 189ms/step - loss: 0.0177 - acc: 0.9979
Epoch 3/10
66/66 [==============================] - 13s 190ms/step - loss: 0.0080 - acc: 0.9998
Epoch 4/10
66/66 [==============================] - 12s 188ms/step - loss: 0.0049 - acc: 0.9998
Epoch 5/10
66/66 [==============================] - 12s 188ms/step - loss: 0.0128 - acc: 0.9974
Epoch 6/10
66/66 [==============================] - 12s 189ms/step - loss: 0.0047 - acc: 0.9991
Epoch 7/10
66/66 [==============================] - 12s 188ms/step - loss: 0.0032 - acc: 0.9996
Epoch 8/10
66/66 [==============================] - 13s 190ms/step - loss: 0.0020 - acc: 0.9995
Epoch 9/10
66/66 [==============================] - 12s 189ms/step - loss: 9.9836e-04 - acc: 1.0000
Epoch 10/10
66/66 [==============================] - 12s 181ms/step - loss: 8.1027e-